In [1]:
!pip install fetch-my-weather
!pip install hands-on-aib

ERROR: Could not find a version that satisfies the requirement hands-on-aib (from versions: none)
ERROR: No matching distribution found for hands-on-aib


In [2]:
# Plotly for different system
# --- Global imports ---
import sys
import plotly.io as pio

# ✅ Plotly 渲染器自动兼容设置（放在所有绘图代码之前）
if "google.colab" in sys.modules:
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "notebook_connected"   

# 之后再 import 其他模块
import plotly.express as px

import os
import requests
import pandas as pd
import importlib

# Graphs
import matplotlib.pyplot as plt
import seaborn as sns

# Chatbot
import re
import nltk

# UI design
from ipywidgets import VBox, HBox, Dropdown, Button, Output
import pyinputplus as pyip
from IPython.display import display, clear_output
import ipywidgets as W
import ipywidgets as widgets
from IPython.display import display, HTML

# Data
from fetch_my_weather import get_weather
from datetime import datetime
from functools import lru_cache
from typing import List, Iterable

## 🌤️ Weather Data Functions

In [3]:
def get_weather_data(location, forecast_days=5):
    """
    Retrieve weather data for a specified location (1–5 day forecast).

    Args:
        location (str): City or location name
        forecast_days (int): Number of days to forecast (1–5)

    Returns:
        dict: Weather data including current conditions and forecast
              {
                "current_condition": {...},
                "forecast": pandas.DataFrame
              }
    """

    # ✅ 从自定义库中获取原始数据
    data = get_weather(location)

    # ---- Current Weather ----
    current = data.current_condition[0]
    current_info = {
        "tempC": float(current.temp_C),
        "FeelsLikeC": float(current.FeelsLikeC),
        "humidity": int(current.humidity),
        "uvIndex": int(current.uvIndex),
        "windSpeedKmph": int(current.windspeedKmph),
        "pressure": int(current.pressure),
        "desc": current.weatherDesc[0].value,
    }

    # ---- Forecast Weather ----
    daily_records = []
    for day in data.weather[:forecast_days]:
        # 取中午12点的天气描述，最接近白天情况
        midday_desc = day.hourly[4].weatherDesc[0].value if day.hourly else "N/A"

        record = {
            "date": day.date,
            "maxTempC": float(day.maxtempC),
            "minTempC": float(day.mintempC),
            "avgTempC": float(day.avgtempC),
            "uvIndex": int(day.uvIndex),
            "sunHour": float(day.sunHour),
            "weatherDesc": midday_desc
        }
        daily_records.append(record)

    forecast_df = pd.DataFrame(daily_records)

    # ✅ 返回结构体（current + forecast）
    return {
        "current_condition": current_info,
        "forecast": forecast_df
    }

In [4]:
# Heplers for the unified the unit. 
def _to_float(x, default=pd.NA):
    try:
        return float(x)
    except (TypeError, ValueError):
        return default

def _val_or(default, *path):
        try:
            obj = path[0]
            for p in path[1:]:
                obj = getattr(obj, p)
            return obj if obj not in (None, "") else default
        except Exception:
            return default
        
def _first_value(lst, default=""):
        # lists like [WeatherDesc(value='Sunny')]
        try:
            return getattr(lst[0], "value")
        except Exception:
            return default
        
def _uv_level(uvi: float) -> str:
        if uvi is None:
            return "N/A"
        if uvi < 3:  return "Low"
        if uvi < 6:  return "Moderate"
        if uvi < 8:  return "High"
        if uvi < 11: return "Very High"
        return "Extreme"
    
def _beaufort(kmph: float) -> str:
        if kmph is None:
            return "N/A"
        # thresholds ~ Beaufort (km/h)
        bft = [
            (1, "0 Calm"), (6, "1 Light Air"), (12, "2 Light Breeze"),
            (20, "3 Gentle Breeze"), (29, "4 Moderate Breeze"),
            (39, "5 Fresh Breeze"), (50, "6 Strong Breeze"),
            (62, "7 Near Gale"), (75, "8 Gale"), (89, "9 Severe Gale"),
            (103, "10 Storm"), (118, "11 Violent Storm")
        ]
        for th, label in bft:
            if kmph <= th:
                return label
        return "12 Hurricane"
    
def _dew_point_c(t_c: float, rh: float):
        # Magnus approximation; returns None if inputs missing
        t = _to_float(t_c, None); h = _to_float(rh, None)
        if t is None or h is None:
            return None
        # constants for water over 0–60°C
        a, b = 17.62, 243.12
        import math
        try:
            gamma = (a * t) / (b + t) + math.log(max(1e-6, h/100.0))
            return round((b * gamma) / (a - gamma), 1)
        except Exception:
            return None

In [5]:
# Detailed current weather inforamtion

def get_current_weather(city: str, grouped: bool = False) -> dict:
    """
    Fetch and return a display-ready dict of current weather for `city`.
    If grouped=True, return a sectioned dict for card-based rendering.
    """
    raw = get_weather(city)
    cc = raw.current_condition[0]
    area = raw.nearest_area[0]
    today = raw.weather[0] if getattr(raw, "weather", None) else None
    astro = today.astronomy[0] if (today and getattr(today, "astronomy", None)) else None

    # ---------- wind formatting ----------
    wind_kmph = _val_or(None, cc, "windspeedKmph")
    wind_dir  = _val_or(None, cc, "winddir16Point")
    wind_deg  = _val_or(None, cc, "winddirDegree")
    if wind_kmph and wind_dir and wind_deg:
        wind = f"{wind_kmph} km/h {wind_dir} ({wind_deg}°) • {_beaufort(_to_float(wind_kmph, None))}"
    elif wind_kmph and wind_dir:
        wind = f"{wind_kmph} km/h {wind_dir} • {_beaufot(_to_float(wind_kmph, None))}"
    elif wind_kmph:
        wind = f"{wind_kmph} km/h • {_beaufort(_to_float(wind_kmph, None))}"
    else:
        wind = "N/A"

    # ---------- computed extras ----------
    temp_c   = _to_float(_val_or(None, cc, "temp_C"), None)
    feels_c  = _to_float(_val_or(None, cc, "FeelsLikeC"), None)
    rh       = _to_float(_val_or(None, cc, "humidity"), None)
    uvi      = _to_float(_val_or(None, cc, "uvIndex"), None)

    feels_delta = None if (temp_c is None or feels_c is None) else round(feels_c - temp_c, 1)
    dew_c       = _dew_point_c(temp_c, rh)  # may be None

    # ---------- flat dict (display-ready with units) ----------
    data = {
        "City": city,
        "Date & Time": _val_or("N/A", cc, "localObsDateTime"),

        "Suburb":  _first_value(_val_or([], area, "areaName"), ""),
        "State":   _first_value(_val_or([], area, "region"), "N/A"),
        "Country": _first_value(_val_or([], area, "country"), "N/A"),
        "Latitude":  _val_or("N/A", area, "latitude"),
        "Longitude": _val_or("N/A", area, "longitude"),

        "Temperature (°C)": None if temp_c is None  else f"{temp_c:.1f} °C",
        "Feels Like (°C)":  None if feels_c is None else f"{feels_c:.1f} °C",
        "Feels Delta (°C)": None if feels_delta is None else f"{feels_delta:+.1f}",
        "Weather Description": _first_value(_val_or([], cc, "weatherDesc"), ""),
        "Weather Code": _val_or("N/A", cc, "weatherCode"),

        "Humidity (%)":   None if rh is None else f"{rh:.0f}%",
        "Dew Point (°C)": None if dew_c is None else f"{dew_c:.1f} °C",
        "Pressure (hPa)": None if _to_float(_val_or(None, cc, "pressure"), None) is None
                           else f"{_to_float(getattr(cc, 'pressure', None)):.0f}",
        "Cloud Cover (%)": None if _to_float(_val_or(None, cc, "cloudcover"), None) is None
                           else f"{_to_float(getattr(cc, 'cloudcover', None)):.0f}%",
        "Visibility (km)": None if _to_float(_val_or(None, cc, "visibility"), None) is None
                           else f"{_to_float(getattr(cc, 'visibility', None)):.0f} km",

        "Wind": wind,
        "Precip (mm)": None if _to_float(_val_or(None, cc, "precipMM"), None) is None
                        else f"{_to_float(getattr(cc, 'precipMM', None)):.1f} mm",

        "UV Index": "N/A" if uvi is None else f"{uvi:.0f} ({_uv_level(uvi)})",
        "Sunrise":  _val_or("N/A", astro, "sunrise") if astro else "N/A",
        "Sunset":   _val_or("N/A", astro, "sunset")  if astro else "N/A",
    }

    # Clean up the NA
    data = {k: v for k, v in data.items() if v is not None}

    if not grouped:
        return data

In [6]:
# ---- ☁️ Brief current weather ----
def brief_current_weather(city: str):
    """Return a one-row DataFrame with Temperature, Feels Like, Condition, and UV Index."""
    raw = get_weather(city)

    # 🩵 如果 get_weather 返回字符串，则解析为 JSON
    if isinstance(raw, str):
        import json
        try:
            raw = json.loads(raw)
        except Exception as e:
            raise ValueError(f"Invalid response format: {e}")

    # 🩵 如果是字典结构（常见于 API JSON 返回）
    if isinstance(raw, dict):
        # 常见结构: {'data': {...}} 或 直接包含 'current_condition'
        if "data" in raw:
            raw = raw["data"]
        if "current_condition" in raw:
            cc = raw["current_condition"][0]
        else:
            raise ValueError("Missing 'current_condition' key in weather data.")
    else:
        # 保留原始对象模式的兼容逻辑
        cc = raw.current_condition[0]

    # ---- Extract UV Index ----
    uv = None
    if hasattr(cc, "uvIndex"):
        uv = float(cc.uvIndex)
    elif hasattr(cc, "UVIndex"):
        uv = float(cc.UVIndex)
    elif isinstance(cc, dict) and "uvIndex" in cc:
        uv = float(cc["uvIndex"])

    # ---- Build display DataFrame ----
    df = pd.DataFrame({
        "Temperature (°C)": [float(cc["temp_C"]) if isinstance(cc, dict) else float(cc.temp_C)],
        "Feels Like (°C)": [float(cc["FeelsLikeC"]) if isinstance(cc, dict) else float(cc.FeelsLikeC)],
        "Condition": [cc["weatherDesc"][0]["value"] if isinstance(cc, dict) else cc.weatherDesc[0].value],
        "UV Index": [uv]
    })

    return df

In [7]:
# Hourly data information
def hourly_weather_data(weather_data):
    """
    从 DailyForecast 对象中提取逐小时的气温、体感温度、湿度和紫外线指数
    :param weather_data: list[DailyForecast]
    :return: pandas.DataFrame(time, tempC, FeelsLikeC, humidity, uvIndex)
    """
    hourly_records = []

    for hour in weather_data[0].hourly:  # 默认取当天
        record = {
            "time": f"{int(hour.time)//100:02d}:00",   # e.g. 600 -> 06:00
            "tempC": float(hour.tempC),
            "FeelsLikeC": float(hour.FeelsLikeC),
            "humidity": int(hour.humidity),
            "uvIndex": int(hour.uvIndex)
        }
        hourly_records.append(record)

    df = pd.DataFrame(hourly_records)
    df["time"] = pd.Categorical(df["time"], categories=df["time"], ordered=True)
    return df

## 📊 Visualisation Functions

In [8]:
# Visualise the detailed current data into a better presentation

def group_weather_data(data: dict) -> dict:
    """
    Group a flat weather data dict (from get_current_weather)
    into logical sections for card-style visualisation.

    Parameters
    ----------
    data : dict
        The flat dictionary returned by get_current_weather().

    Returns
    -------
    dict
        A grouped dictionary, where keys are section titles and
        values are sub-dicts containing label-value pairs.
    """

    sections = {
        "Now": {
            "Temperature (°C)": data.get("Temperature (°C)"),
            "Feels Like (°C)":  data.get("Feels Like (°C)"),
            "Feels Delta (°C)": data.get("Feels Delta (°C)"),
            "Weather Description": data.get("Weather Description"),
            "Weather Code": data.get("Weather Code"),
        },
        "Air": {
            "Humidity (%)": data.get("Humidity (%)"),
            "Dew Point (°C)": data.get("Dew Point (°C)"),
            "Pressure (hPa)": data.get("Pressure (hPa)"),
            "Cloud Cover (%)": data.get("Cloud Cover (%)"),
            "Visibility (km)": data.get("Visibility (km)"),
            "Precip (mm)": data.get("Precip (mm)"),
        },
        "Wind": {
            "Wind": data.get("Wind"),
        },
        "Sun & UV": {
            "UV Index": data.get("UV Index"),
            "Sunrise":  data.get("Sunrise"),
            "Sunset":   data.get("Sunset"),
        },
        "Location": {
            "City": data.get("City"),
            "Suburb": data.get("Suburb"),
            "State": data.get("State"),
            "Country": data.get("Country"),
            "Latitude": data.get("Latitude"),
            "Longitude": data.get("Longitude"),
            "Date & Time": data.get("Date & Time"),
        },
    }

    # Clean up the NA
    def _clean(d: dict) -> dict:
        return {k: v for k, v in d.items() if v not in (None, "N/A", "")}

    sections = {title: _clean(items) for title, items in sections.items() if _clean(items)}
    return sections

In [9]:
def _render_detail_cards(sections: dict, framework: str = "streamlit", **kwargs):
    """
    🌈 Render grouped weather data in a gradient soft-glass purple theme (2-column layout).
    """
    from IPython.display import HTML, display

    # 💜 Section 配色 & 图标
    section_colors = {
        "Now": "#f3e9ff",
        "Air": "#efe6ff",
        "Wind": "#f2e8ff",
        "Sun & UV": "#f7f0ff",
        "Location": "#f1eaff"
    }
    title_icons = {
        "Now": "🌤️",
        "Air": "🌫️",
        "Wind": "🌬️",
        "Sun & UV": "☀️",
        "Location": "📍"
    }

    # 💅 样式定义：两列卡片 + 渐变柔光背景
    style_html = """
    <style>
    .detail-wrapper {
        background: linear-gradient(180deg, #f7f3ff 0%, #ffffff 90%);
        border-radius: 20px;
        padding: 30px 35px 40px 35px;
        width: 96%;
        max-width: 1300px;
        margin: 25px auto;
        box-shadow: 0 6px 14px rgba(140, 100, 255, 0.08);
        font-family: 'Segoe UI','Helvetica Neue',sans-serif;
    }

    /* 🌸 两列自适应布局 */
    .detail-grid {
        display: grid;
        grid-template-columns: repeat(auto-fill, minmax(500px, 1fr));
        gap: 25px;
        justify-items: center;
    }

    .detail-card {
        background-color: rgba(255, 255, 255, 0.8);
        backdrop-filter: blur(6px);
        border-radius: 18px;
        box-shadow: 0 3px 10px rgba(130, 90, 255, 0.12);
        padding: 20px 28px;
        width: 100%;
        max-width: 520px;
        color: #3a2c5f;
        transition: transform 0.25s ease, box-shadow 0.25s ease;
    }
    .detail-card:hover {
        transform: scale(1.02);
        box-shadow: 0 6px 14px rgba(120, 80, 220, 0.25);
    }

    .detail-card-title {
        font-size: 18px;
        font-weight: 650;
        color: #5b3cc4;
        margin-bottom: 10px;
        background-color: rgba(155,130,255,0.1);
        padding: 6px 12px;
        border-radius: 8px;
        display: inline-block;
        box-shadow: inset 0 1px 2px rgba(255,255,255,0.5);
    }

    .detail-row {
        display: flex;
        justify-content: space-between;
        margin: 5px 0;
        font-size: 15px;
    }
    .detail-row span:first-child {
        font-weight: 500;
        color: #45307e;
    }
    .detail-row span:last-child {
        color: #2e1f55;
        font-weight: 550;
    }
    </style>
    """

    # 💜 渲染每个卡片
    cards_html = ""
    for title, kv in sections.items():
        if not kv:
            continue
        bg = section_colors.get(title, "#f3e8ff")
        icon = title_icons.get(title, "")
        rows = "".join(
            f"<div class='detail-row'><span>{k}</span><span>{v}</span></div>"
            for k, v in kv.items()
        )
        cards_html += f"""
        <div class="detail-card" style="background:{bg};">
            <div class="detail-card-title">{icon} {title}</div>
            {rows}
        </div>
        """

    html = f"""
    {style_html}
    <div class="detail-wrapper">
        <div class="detail-grid">
            {cards_html}
        </div>
    </div>
    """

    display(HTML(html))

In [10]:
def visualisation_hourly_weather(df):
    import plotly.express as px
    import ipywidgets as widgets
    from IPython.display import display, clear_output

    # --- 输出容器 ---
    plot_output = widgets.Output()

    # --- 绘图函数 ---
    def draw_chart(metric):
        with plot_output:
            clear_output(wait=True)
            # 🎨 配色与标题映射
            color_map = {
                "tempC": "#7b5ce1",
                "FeelsLikeC": "#c57aff",
                "humidity": "#3cc47b",
                "uvIndex": "#f6b93b"
            }
            title_map = {
                "tempC": "🌡️ Hourly Temperature (°C)",
                "FeelsLikeC": "🌀 Hourly Feels Like (°C)",
                "humidity": "💧 Hourly Humidity (%)",
                "uvIndex": "☀️ Hourly UV Index"
            }

            fig = px.line(
                df,
                x="time",
                y=metric,
                markers=True,
                title=title_map[metric],
                color_discrete_sequence=[color_map[metric]],
                hover_data={"time": True, metric: ':.1f'}
            )

            fig.update_traces(line=dict(width=3),
                              marker=dict(size=8, line=dict(width=1.5, color='white')))

            fig.update_layout(
                template="plotly_white",
                plot_bgcolor="#f9f7ff",
                paper_bgcolor="white",
                title_font=dict(size=18, color="#5b3cc4", family="Segoe UI"),
                font=dict(color="#4b3ca8", family="Segoe UI"),
                hoverlabel=dict(bgcolor="#f2eaff", font_size=13, font_family="Segoe UI"),
                margin=dict(l=50, r=30, t=60, b=40),
            )
            fig.update_xaxes(title="Time", showgrid=True, gridcolor="#e0d7ff")
            fig.update_yaxes(title=metric, showgrid=True, gridcolor="#e0d7ff")

            fig.show()  # ✅ 强制渲染

    # --- 右侧 Metric 卡片（竖列）---
    toggle = widgets.ToggleButtons(
        options=[
            ("🌡️ Temperature", "tempC"),
            ("🌀 Feels Like", "FeelsLikeC"),
            ("💧 Humidity", "humidity"),
            ("☀️ UV Index", "uvIndex")
        ],
        value="tempC",
        layout=widgets.Layout(
            flex_flow='column',
            align_items='stretch',
            width='220px'
        ),
        style={'font_weight': 'bold'}
    )

    # --- 样式注入（紫色立体风）---
    style_html = widgets.HTML("""
    <style>
    .jupyter-widgets.widget-toggle-buttons {
        background-color: transparent !important;
        display: flex !important;
        flex-direction: column !important;
        gap: 10px !important;
    }
    .jupyter-widgets.widget-toggle-buttons .widget-toggle-button {
        background-color: #ede2ff !important;
        color: #4b0082 !important;
        border: 2px solid #cbb3ff !important;
        border-radius: 12px !important;
        font-weight: 600 !important;
        box-shadow: 0 3px 8px rgba(130, 100, 230, 0.15) !important;
        transition: all 0.25s ease-in-out !important;
        padding: 6px 10px !important;
        text-align: left !important;
    }
    .jupyter-widgets.widget-toggle-buttons .widget-toggle-button:hover {
        background-color: #c9b3ff !important;
        color: white !important;
        transform: translateY(-2px);
    }
    .jupyter-widgets.widget-toggle-buttons .widget-toggle-button.mod-active {
        background-color: #8462ff !important;
        color: white !important;
        border-color: #a68aff !important;
        box-shadow: 0 3px 8px rgba(90, 60, 200, 0.3) !important;
    }
    </style>
    """)

    # --- 切换逻辑 ---
    def on_change(change):
        if change["name"] == "value" and change["new"] != change["old"]:
            draw_chart(change["new"])

    toggle.observe(on_change)

    # --- 布局：左图 + 右卡片 ---
    metric_card = widgets.VBox([
        widgets.HTML("<div style='font-weight:700;color:#5b3cc4;margin-bottom:8px;'>📊 Select Metric</div>"),
        toggle
    ])
    visual_box = widgets.HBox([
        plot_output,
        widgets.VBox([style_html, metric_card],
                     layout=widgets.Layout(margin='0 0 0 25px'))  # 右侧留白
    ])

    # --- 初始显示 ---
    draw_chart("tempC")
    display(visual_box)

In [19]:
def show_forecast_weather(location):
    """
    🌈 Balanced Forecast Weather layout
    - Perfect full-width fit (no horizontal bar)
    - Visually consistent with Current Weather visualisation
    """
    import plotly.express as px
    import time
    from IPython.display import display, HTML, clear_output

    # ---- 🌀 Loading 动画 ----
    display(HTML("""
    <div id='loading-forecast' style='text-align:center; margin-top:40px; font-family:Segoe UI; color:#5b3cc4;'>
        <div style='display:inline-block; width:22px; height:22px;
                    border:3px solid #d8c6ff; border-top:3px solid #5b3cc4;
                    border-radius:50%; animation:spin 1s linear infinite;
                    margin-right:8px; vertical-align:middle;'></div>
        <b>Loading forecast data...</b>
        <style>@keyframes spin {0%{transform:rotate(0deg);}100%{transform:rotate(360deg);}}</style>
    </div>
    """))
    time.sleep(1.0)

    try:
        weather_data = get_weather_data(location)
        df_forecast = weather_data["forecast"]

        clear_output(wait=True)

        # ---- 💅 响应式布局样式 ----
        style_html = """
        <style>
        .forecast-title {
            text-align: center;
            font-weight: 700;
            font-size: 20px;
            color: #5b3cc4;
            margin: 20px 0 10px 0;
            text-shadow: 0 1px 3px rgba(140,100,255,0.25);
        }

        .forecast-container {
            display: flex;
            flex-wrap: wrap;
            justify-content: center;
            background-color: #f7f3ff;
            border-radius: 18px;
            padding: 18px 25px;
            width: 95%;
            margin: 15px auto 25px auto;
            box-shadow: 0 4px 12px rgba(140, 100, 255, 0.1);
            font-family: 'Segoe UI', 'Helvetica Neue', sans-serif;
            max-width: 1300px;
        }

        .forecast-card {
            background-color: white;
            border-radius: 15px;
            padding: 16px 20px;
            margin: 10px;
            text-align: center;
            width: 220px;
            box-shadow: 0 4px 10px rgba(120, 80, 220, 0.12);
            transition: transform 0.25s ease, box-shadow 0.25s ease;
        }
        .forecast-card:hover {
            transform: scale(1.04);
            box-shadow: 0 6px 14px rgba(90, 60, 200, 0.25);
        }

        .forecast-date {
            font-weight: 650;
            font-size: 16px;
            color: #5b3cc4;
            margin-bottom: 5px;
        }
        .forecast-temp {
            font-size: 15px;
            font-weight: 600;
            margin: 4px 0;
            color: #3a2c6b;
        }
        .forecast-desc {
            font-style: italic;
            color: #6b5b95;
            margin-top: 6px;
            font-size: 14px;
        }
        </style>
        """

        # ---- 🧩 卡片内容 ----
        card_html = "".join([
            f"""
            <div class="forecast-card">
                <div class="forecast-date">{row.date}</div>
                <div class="forecast-temp">🌡️ {row.maxTempC}°C / {row.minTempC}°C</div>
                <div>☀️ UV: {row.uvIndex} | 🌞 {row.sunHour}h</div>
                <div class="forecast-desc">{row.weatherDesc}</div>
            </div>
            """ for _, row in df_forecast.iterrows()
        ])

        # ---- 📋 标题 + 卡片 ----
        display(HTML(style_html + f"""
        <div class="forecast-title">📅 3-Day Weather Forecast — {location}</div>
        <div class="forecast-container">{card_html}</div>
        """))

        # ---- 📈 Plotly 图 ----
        df_plot = df_forecast.melt(
            id_vars=["date"],
            value_vars=["maxTempC", "minTempC"],
            var_name="Type", value_name="Temperature (°C)"
        )
        df_plot["Type"] = df_plot["Type"].replace({
            "maxTempC": "Max Temp (°C)",
            "minTempC": "Min Temp (°C)"
        })

        fig = px.line(
            df_plot,
            x="date",
            y="Temperature (°C)",
            color="Type",
            markers=True,
            color_discrete_map={
                "Max Temp (°C)": "#7b5ce1",
                "Min Temp (°C)": "#ffb347"
            },
            hover_name="Type",
            hover_data={"date": True, "Temperature (°C)": ':.1f'},
            title=f"🌤️ Temperature Trend — {location}"
        )

        fig.update_traces(line=dict(width=2.8),
                          marker=dict(size=7, line=dict(width=1.3, color='white')))

        fig.update_layout(
            template="plotly_white",
            plot_bgcolor="#f9f7ff",
            paper_bgcolor="white",
            title_font=dict(size=18, color="#5b3cc4", family="Segoe UI"),
            font=dict(color="#4b3ca8", family="Segoe UI"),
            hoverlabel=dict(bgcolor="#f2eaff", font_size=12, font_family="Segoe UI"),
            legend=dict(title=None, orientation="h", y=1.1, x=0.25),
            margin=dict(l=40, r=30, t=70, b=40),
            width=1000,
            height=440
        )

        fig.update_xaxes(showgrid=True, gridwidth=0.5, gridcolor="#e0d7ff", tickangle=-25)
        fig.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor="#e0d7ff")

        display(fig)

    except Exception as e:
        clear_output(wait=True)
        display(HTML(f"<span style='color:red;'>❌ Unable to load forecast data: {e}</span>"))

## 🤖 Natural Language Processing

In [12]:
# NLP
KNOWN_CITIES = [
    "Perth, Australia", "Sydney, Australia", "Melbourne, Australia", "Brisbane, Australia",
    "Adelaide, Australia", "Canberra, Australia", "Hobart, Australia", "Darwin, Australia",
    "Auckland, New Zealand", "Wellington, New Zealand", "Christchurch, New Zealand",
    "Singapore, Singapore", "Seoul, South Korea", "San Francisco, USA", "Seattle, USA",
    "Sapporo, Japan", "Stockholm, Sweden", "Santiago, Chile", "São Paulo, Brazil",
    "Shanghai, China", "Shenzhen, China", "Shenyang, China", "Suzhou, China", "Nanchang, China",
    "Ganzhou, China"
]

def parse_city(text: str) -> str | None:
    t = text.lower()
    for c in KNOWN_CITIES:
        if c.lower() in t:
            return c
    for c in KNOWN_CITIES:
        name = c.split(",")[0].strip().lower()
        if re.search(rf"\b{re.escape(name)}\b", t):
            return c
    return None

def parse_need(text: str) -> str:
    """
    解析需求类别：'uv' / 'humidity' / 'wind' / 'pressure' / 'brief' / 'detail'
    """
    t = text.lower()
    if "uv" in t or "紫外" in t:
        return "uv"
    if "湿度" in t or "humidity" in t:
        return "humidity"
    if "风" in t or "wind" in t:
        return "wind"
    if "压" in t or "pressure" in t or "气压" in t:
        return "pressure"
    if "详细" in t or "detail" in t or "more" in t:
        return "detail"
    # 兜底：简报
    return "brief"

## 🧭 User Interface

In [20]:
from IPython.display import HTML, display

style_html = """
<style>
/* 💜 全局紫色柔和主题 —— 支持 ipywidgets 8+ */
body, .widget-label, .widget-html-content {
    font-family: 'Segoe UI', 'Helvetica Neue', sans-serif !important;
}

/* 顶部区域 */
.top-section {
    background-color: #f3e8ff;
    border-radius: 15px;
    padding: 15px 20px;
    box-shadow: 0 3px 10px rgba(130, 90, 255, 0.15);
    margin-bottom: 20px;
}
.title {
    color: #5b3cc4;
    font-size: 22px;
    font-weight: 650;
    margin-bottom: 10px;
}

/* ---- 按钮样式（Current/Forecast Weather） ---- */
.widget-button {
    background-color: #e9dbff !important;
    color: #4b0082 !important;
    border-radius: 10px !important;
    border: 2px solid #c3aaff !important;
    font-weight: 600 !important;
    transition: all 0.25s ease !important;
}
.widget-button:hover {
    background-color: #c9b3ff !important;
    color: white !important;
    transform: translateY(-2px);
}
.widget-button.mod-active {
    background-color: #8462ff !important;
    color: white !important;
}

/* ---- 紫色 ToggleButtons 样式（去掉背景盒子） ---- */
.widget-toggle-buttons,
.widget-toggle-buttons > div[role="group"] {
    background-color: transparent !important;  /* ✅ 移除淡紫背景 */
    border-radius: 0 !important;              /* ✅ 去掉圆角 */
    padding: 0 !important;                    /* ✅ 去掉内边距 */
    display: flex !important;
    flex-direction: column !important;
    gap: 10px !important;
}
.widget-toggle-buttons button {
    background-color: #ede2ff !important;
    color: #4b0082 !important;
    border: 2px solid #cbb3ff !important;
    border-radius: 10px !important;
    font-weight: 600 !important;
    box-shadow: 0 2px 6px rgba(100, 70, 200, 0.15) !important;
    transition: all 0.25s ease !important;
}
.widget-toggle-buttons button:hover {
    background-color: #c9b3ff !important;
    color: white !important;
}
.widget-toggle-buttons button.mod-active {
    background-color: #8462ff !important;
    color: white !important;
    border-color: #a68aff !important;
}

/* ---- Tab 样式（Description / Visualisation） ---- */
.widget-tab > .p-TabBar-tab {
    background-color: #f1e8ff !important;
    color: #4b0082 !important;
    border-radius: 8px !important;
    font-weight: 600 !important;
    margin-right: 4px !important;
    transition: all 0.25s ease-in-out !important;
}
.widget-tab > .p-TabBar-tab:hover {
    background-color: #c9b3ff !important;
    color: white !important;
}
.widget-tab > .p-TabBar-tab.p-mod-current {
    background-color: #8462ff !important;
    color: white !important;
    box-shadow: 0 2px 6px rgba(90, 60, 200, 0.25) !important;
}

/* ---- Metric Card 标题 ---- */
.metric-card h4 {
    color: #5b3cc4;
    font-weight: 700;
    font-size: 16px;
    text-align: center;
    margin-bottom: 12px;
}
</style>
"""
display(HTML(style_html))

In [27]:
# ---- 🌤️ Top Section ----
title = widgets.HTML("<div class='title'>🌤️ Weather Advisor — Control Panel</div>")
subtitle = widgets.HTML("<div class='subtitle'>Select a function below to explore weather data or chat with our assistant.</div>")

# ---- 🌸 Two main buttons ----
btn_current = widgets.Button(
    description='Current Weather',
    layout=widgets.Layout(width='150px', height='35px'),
    style={'button_color': '#c7a4ff', 'font_weight': 'bold'}
)
btn_forecast = widgets.Button(
    description='Forecast Weather',
    layout=widgets.Layout(width='150px', height='35px'),
    style={'button_color': '#c7a4ff', 'font_weight': 'bold'}
)

# ---- 🏙️ Entering city ----
city_input = widgets.Text(
    placeholder='Enter a city name (e.g. Sydney, Australia)',
    description='City:',
    style={'description_width': '60px'},
    layout=widgets.Layout(width='300px')
)
confirm_btn = widgets.Button(
    description='Confirm',
    layout=widgets.Layout(width='100px', height='35px'),
    style={'button_color': '#b388ff', 'font_weight': 'bold'}
)
city_section = widgets.HBox([city_input, confirm_btn])
city_container = widgets.VBox([], layout=widgets.Layout(margin='20px 0 0 0'))

# ---- 输出区 ----
output = widgets.Output()
brief_output = widgets.Output()
details_output = widgets.Output()

# ---- 控制逻辑 ----
selected_action = widgets.Text(value='', layout=widgets.Layout(display='none'))

# ✅ 绑定改进后的按钮逻辑
def show_city_input(btn):
    """
    🌈 改进版：支持从 Current Weather 跳转到 Forecast Weather。
    """
    city_container.children = [city_section]
    selected_action.value = btn.description

    with output:
        clear_output(wait=True)
        display(HTML(f"<b style='color:#5b3cc4;'>You selected:</b> {btn.description}"))

    # 🌤️ 如果当前输入框中已有城市，则自动加载相应界面
    city = city_input.value.strip()
    if city:
        if btn.description == "Forecast Weather":
            # 自动跳转到 forecast 界面
            with output:
                clear_output(wait=True)
            on_confirm_click(None)
        elif btn.description == "Current Weather":
            # 自动跳转到 current weather 界面
            with output:
                clear_output(wait=True)
            on_confirm_click(None)

# ✅ 绑定改进后的按钮逻辑
btn_current.on_click(show_city_input)
btn_forecast.on_click(show_city_input)


# ---- 当点击 Confirm ----
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
import time

# 🟣 确保输出区是全局定义的
output = widgets.Output()
brief_output = widgets.Output()
details_output = widgets.Output()

def on_confirm_click(_):
    """
    🌈 Unlimited search version — supports continuous city queries
    without losing output binding.
    """
    from IPython.display import HTML, clear_output
    import time

    city = city_input.value.strip()
    action = selected_action.value

    if not city:
        with output:
            clear_output(wait=True)
            display(HTML("<span style='color:red;'>⚠️ Please enter a city name first.</span>"))
        return

    # 🌸 Step 1. 清空输出区内容但保留绑定关系
    with brief_output:
        clear_output(wait=True)
    with details_output:
        clear_output(wait=True)
    with output:
        clear_output(wait=True)
        display(brief_output, details_output)  # 再次挂载输出区，保持活性

    # 🌈 Step 2. 加载提示
    with brief_output:
        display(HTML(f"""
        <div style='text-align:center; margin-top:20px; font-family:Segoe UI; color:#5b3cc4;'>
            <div style='display:inline-block; width:22px; height:22px;
                        border:3px solid #d8c6ff; border-top:3px solid #5b3cc4;
                        border-radius:50%; animation:spin 1s linear infinite;
                        margin-right:8px; vertical-align:middle;'></div>
            <b>Loading weather data for <span style="color:#7b5ce1;">{city}</span>...</b>
            <style>@keyframes spin {{0% {{transform: rotate(0deg);}} 100% {{transform: rotate(360deg);}}}}</style>
        </div>
        """))
    time.sleep(1.0)

    # 🌤️ Step 3. 加载天气数据
    with brief_output:
        clear_output(wait=True)
        try:
            if action == "Current Weather":
                df = brief_current_weather(city)

                # 标题卡片
                display(HTML(f"""
                <div style='background:#f8f3ff; border-radius:15px;
                    padding:18px 25px; margin-top:10px; box-shadow:0 3px 10px rgba(140,100,255,0.1);'>
                    <div style='font-weight:bold; color:#5b3cc4; font-size:18px; margin-bottom:10px;'>
                        📍 {city}
                    </div>
                """))

                # 数据输出
                for col, val in df.iloc[0].items():
                    display(HTML(f"<div style='margin:4px 0;'><b>{col}:</b> {val}</div>"))
                display(HTML("</div><br>"))
                display(show_details_btn)

            elif action == "Forecast Weather":
                show_forecast_weather(city)

            else:
                display(HTML("<i style='color:gray;'>⚙️ Unknown action.</i>"))

        except Exception as e:
            display(HTML(f"<span style='color:red;'>❌ Unable to fetch data: {e}</span>"))

# 🔄 Step 4. 始终确保按钮与输入框绑定
confirm_btn.on_click(on_confirm_click)
city_input.on_submit(on_confirm_click)

# ---- 🟣 Show Details ----
show_details_btn = widgets.Button(
    description='Show Detailed Weather Information',
    layout=widgets.Layout(width='300px', height='35px', margin='10px 0 0 0'),
    style={'button_color': '#d1b3ff', 'font_weight': 'bold'}
)

def show_detailed_weather(_):
    """
    Display detailed weather info with full-width Description layout and unified styling.
    """
    import time
    from IPython.display import display, HTML, clear_output
    import ipywidgets as widgets

    with details_output:
        clear_output(wait=True)
        city = city_input.value.strip()
        if not city:
            display(HTML("<span style='color:red;'>⚠️ Please enter a city name first.</span>"))
            return

        # 🌀 Loading 动画
        display(HTML(f"""
        <div style='text-align:center; margin-top:20px; font-family:Segoe UI; color:#5b3cc4;'>
          <div style='display:inline-block; width:25px; height:25px;
                      border:3px solid #d8c6ff; border-top:3px solid #5b3cc4;
                      border-radius:50%; animation:spin 1s linear infinite; margin-right:10px;'>
          </div>
          <b>Loading detailed weather for <span style='color:#7b5ce1'>{city}</span>...</b>
          <style>
          @keyframes spin {{ 0% {{ transform: rotate(0deg); }} 100% {{ transform: rotate(360deg); }} }}
          </style>
        </div>
        """))
        time.sleep(1.2)

        try:
            # 🧩 获取天气数据
            data = get_current_weather(city)
            grouped = group_weather_data(data)

            # ✅ 隐藏上一个区域
            brief_output.layout.display = 'none'

            # 🌈 标题
            title_html = widgets.HTML(f"""
            <div style='
                text-align:center;
                font-weight:700;
                font-size:22px;
                color:#5b3cc4;
                font-family:"Segoe UI", Helvetica, sans-serif;
                margin:10px 0 15px 0;
                text-shadow:0 1px 3px rgba(150,100,255,0.2);
            '>
                🌈 Detailed Weather for <span style="color:#5b3cc4;">{city}</span>
            </div>
            """)

            # 🧭 分页按钮
            tab = widgets.ToggleButtons(
                options=[('📄 Description', 'desc'), ('📊 Visualisation', 'vis')],
                value='desc',
                layout=widgets.Layout(justify_content='center', width='auto', margin='8px 15px 0'),
                style={'font_weight': 'bold'}
            )

            # 💅 全新样式（扩大 Description 页宽度 & 统一视觉风格）
            display(widgets.HTML("""
            <style>
            .jupyter-widgets.widget-toggle-buttons {
                display:flex; justify-content:center; gap:10px;
            }
            .jupyter-widgets.widget-toggle-buttons .widget-toggle-button {
                background-color:#f7f1ff;
                border:2px solid #d8c6ff;
                border-radius:10px;
                font-family:'Segoe UI', sans-serif;
                font-size:15px;
                font-weight:600;
                color:#5b3cc4;
                padding:8px 22px;
                box-shadow:0 2px 6px rgba(140,100,255,0.15);
                transition:all 0.25s ease-in-out;
            }
            .jupyter-widgets.widget-toggle-buttons .widget-toggle-button:hover {
                background-color:#e8d9ff;
                transform:translateY(-2px);
            }
            .jupyter-widgets.widget-toggle-buttons .widget-toggle-button.mod-active {
                background-color:#8d6bff !important;
                color:white !important;
                border-color:#8d6bff !important;
                box-shadow:0 4px 10px rgba(130,90,255,0.35);
            }

            /* 🌟 Full-width Description Section */
            .detail-section {
                width: 98% !important;
                max-width: 1600px !important;
                margin: 0 auto !important;
                transition: all 0.3s ease-in-out;
            }

            /* 💜 Weather Card */
            .weather-card {
                background-color:#f3e8ff;
                border-radius:20px;
                padding:25px 40px;
                margin:25px auto;
                width:96%;
                max-width:1500px;
                box-shadow:0 4px 10px rgba(130,90,255,0.15);
            }
            .weather-card h4 {
                margin-bottom:10px;
                font-weight:700;
                color:#5b3cc4;
                font-size:18px;
            }
            .weather-card table {
                width:100%;
                font-size:15px;
                color:#333;
            }
            .weather-card td:last-child {
                text-align:right;
                color:#444;
                font-weight:500;
            }
            </style>
            """))

            # 🧾 Description 页
            desc_output = widgets.Output()
            with desc_output:
                display(HTML("<div class='detail-section'>"))
                _render_detail_cards(grouped)
                display(HTML("</div>"))

            # 📈 Visualisation 页
            vis_output = widgets.Output()
            with vis_output:
                try:
                    raw_data = get_weather(city)
                    if raw_data is None:
                        raise ValueError("No data returned from get_weather().")

                    if hasattr(raw_data, "weather"):
                        weather_data = raw_data.weather
                    elif isinstance(raw_data, dict):
                        possible_keys = [k for k in raw_data.keys() if "weather" in k or "hour" in k]
                        if possible_keys:
                            weather_data = raw_data[possible_keys[0]]
                        else:
                            raise KeyError(f"Cannot find usable 'weather' field. Keys: {list(raw_data.keys())}")
                    else:
                        raise TypeError(f"Unexpected type: {type(raw_data)}")

                    df_today = hourly_weather_data(weather_data)
                    if df_today is None or df_today.empty:
                        raise ValueError("hourly_weather_data() returned empty dataframe.")
                    if "time" not in df_today.columns:
                        raise KeyError(f"'time' column not found. Columns: {list(df_today.columns)}")

                    display(HTML(f"<p style='color:#5b3cc4;'>✅ Loaded {len(df_today)} hourly records for {city}.</p>"))
                    visualisation_hourly_weather(df_today)

                except Exception as e:
                    display(HTML(f"<div style='color:red;'>⚠️ Error generating visualisation: {e}</div>"))

            # 🔄 切换逻辑
            content = widgets.VBox([desc_output])
            def switch(change):
                if change['name'] == 'value':
                    content.children = [desc_output if change['new'] == 'desc' else vis_output]
            tab.observe(switch, names='value')

            # 🎯 显示内容
            details_output.layout.display = 'block'
            with details_output:
                clear_output(wait=True)
                display(widgets.VBox([title_html, tab, content]))

        except Exception as e:
            details_output.clear_output()
            display(HTML(f"<span style='color:red;'>❌ Unable to load detailed weather: {e}</span>"))

show_details_btn.on_click(show_detailed_weather)

# ---- 💬 Smart Weather Chatbot ----
current_city = None
current_need = None
chat_history = []

# ---- 💬 智能版 Chatbot 响应（使用真实天气数据） ----
def chatbot_response(msg: str) -> str:
    global current_city, current_need, chat_history

    chat_history.append(msg)
    msg_lower = msg.lower()

    # --- Step 1: 解析城市 ---
    city = parse_city(msg)
    if city:
        current_city = city

    # --- Step 2: 解析需求 ---
    need = parse_need(msg)
    if need:
        current_need = need

    # --- Step 3: 检查上下文 ---
    if not current_city:
        return "🌍 Which city are you asking about?"
    if not current_need:
        return f"🤔 What do you want to know about {current_city}? (e.g., UV, humidity, wind, or temperature)"

    # --- Step 4: 获取真实数据 ---
    try:
        data = get_weather(current_city)
        cc = data.current_condition[0]
        temp = cc.temp_C
        feels = cc.FeelsLikeC
        humidity = cc.humidity
        pressure = cc.pressure
        uv = cc.uvIndex
        wind = cc.windspeedKmph
        desc = cc.weatherDesc[0].value
    except Exception as e:
        return f"⚠️ Sorry, I couldn’t fetch data for {current_city}. (Error: {e})"

    # --- Step 5: 按需求回复 ---
    if current_need == "uv":
        return f"☀️ In {current_city}, the UV index is {uv}."
    elif current_need == "humidity":
        return f"💧 The humidity in {current_city} is {humidity}%."
    elif current_need == "pressure":
        return f"🌡️ The air pressure in {current_city} is {pressure} hPa."
    elif current_need == "wind":
        return f"🌬️ The wind speed in {current_city} is {wind} km/h."
    elif current_need == "detail":
        return f"📊 {current_city} weather: {desc}, {temp}°C (feels like {feels}°C), humidity {humidity}%, UV {uv}, wind {wind} km/h, pressure {pressure} hPa."
    elif current_need == "brief":
        return f"🌤️ It's {desc.lower()} in {current_city}, around {temp}°C."
    else:
        return f"🤖 In {current_city}: {desc}, {temp}°C, humidity {humidity}%, UV {uv}."

# 🧩 UI 组件设计
chat_output = widgets.Output()

chat_input = widgets.Text(
    placeholder='Ask me about the weather...',
    description='You:',
    style={'description_width': '50px'},
    layout=widgets.Layout(width='400px')
)

send_button = widgets.Button(
    description='Send 💜',
    style={'button_color': '#c7a4ff', 'font_weight': 'bold'},
    layout=widgets.Layout(width='100px')
)

# ✉️ 发送消息逻辑
def send_message(_):
    text = chat_input.value.strip()
    if not text:
        return
    chat_input.value = ""

    with chat_output:
        display(HTML(f"<b style='color:#4b0082;'>You:</b> {text}"))
        reply = chatbot_response(text)
        display(HTML(f"<b style='color:#5b3cc4;'>WeatherBot:</b> {reply}"))

send_button.on_click(send_message)

# 🌈 布局样式
chatbox = widgets.VBox([
    widgets.HTML("<div class='chat-title' style='font-weight:bold;color:#5b3cc4;font-size:16px;margin-bottom:5px;'>💬 Chat with WeatherBot</div>"),
    widgets.HBox([chat_input, send_button]),
    chat_output
])

# ---- 🌟 Combine UI into one main container ----
def main_app():
    button_row = widgets.HBox(
        [btn_current, btn_forecast],
        layout=widgets.Layout(justify_content='flex-start', gap='30px')
    )

    top_section = widgets.VBox([
    title,
    subtitle,
    button_row,
    city_container,
    output  # ✅ 改成显示主输出容器
])
    
    chat_section = chatbox  # 直接用你之前定义好的 chatbox

    # ✅ 整体布局
    app = widgets.VBox([
        top_section,
        chat_section,
        widgets.HTML("<hr style='border:none;height:2px;background:#f3e8ff;margin:15px 0;'>")
    ], layout=widgets.Layout(width='100%', align_items='flex-start'))

    return app

# ✅ 只调用一次
ui = main_app()
display(ui)

C:\Users\angla\AppData\Local\Temp\ipykernel_20492\1820761727.py:154: DeprecationWarning:

on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').



In [15]:
# 之后current weather的city search bottom看看可不可以当user看完了他们想要的数据之后重新在search bottom里面输入，让他们看到新的xity的数据
# current weather show detailed weather的动画载入没有了

In [16]:
# 当进入了current weather页面的时候就不能点击进入forecast weather的界面了，必须要重新run一边cell才能看forecast weather。

In [17]:
# ChatBot部分添加修改，这个chatBot太笨了，没有记忆。

In [18]:
import plotly.express as px
import plotly.io as pio
import pandas as pd

# 确认当前渲染器
print("Current renderer:", pio.renderers.default)

# 简单绘图测试
df = pd.DataFrame({
    "time": ["00:00", "03:00", "06:00", "09:00", "12:00"],
    "tempC": [20, 21, 23, 26, 27]
})

fig = px.line(df, x="time", y="tempC", title="Plotly Rendering Test")
fig.show()

Current renderer: notebook_connected


In [26]:
data = get_weather("Sydney")
data

WeatherResponse(current_condition=[CurrentCondition(FeelsLikeC='25', FeelsLikeF='76', cloudcover='0', humidity='65', localObsDateTime='2025-10-19 07:30 PM', observation_time='08:30 AM', precipInches='0.0', precipMM='0.0', pressure='1012', pressureInches='30', temp_C='22', temp_F='72', uvIndex='0', visibility='10', visibilityMiles='6', weatherCode='113', weatherDesc=[WeatherDesc(value='Sunny')], weatherIconUrl=[WeatherIconUrl(value='')], winddir16Point='NNE', winddirDegree='30', windspeedKmph='29', windspeedMiles='18')], nearest_area=[NearestArea(areaName=[AreaName(value='Kirribilli')], country=[Country(value='Australia')], latitude='-33.850', longitude='151.217', population='3483', region=[Region(value='New South Wales')], weatherUrl=[WeatherIconUrl(value='')])], request=[Request(query='Lat -33.85 and Lon 151.22', type='LatLon')], weather=[DailyForecast(astronomy=[Astronomy(moon_illumination='6', moon_phase='Waning Crescent', moonrise='04:50 AM', moonset='05:09 PM', sunrise='06:09 AM',